# Surreal Numbers
This is a Python module which implements surreal numbers as defined in the [book of the same name](https://omoikane.kwansystems.org/wiki/index.php/Surreal_Numbers) by Knuth. I am trying to work along with Alice and Bob, so the concept of spoliers exists.

## Infinitesimal numbers
Surreal numbers are a superset of the conventional definition of the real numbers. Not only is every real number present, there is also an infinite set of infinitesimal values, which to me is the interesting part of surreal numbers. I want to use them to do differential calculus without bothering with limits.

The real numbers follow the [Archimedean property](https://en.wikipedia.org/wiki/Archimedean_property). Given any two positive real numbers $a$ and $b$, you can find a finite positive integer $n$ such that $na>b$. In other words, you can "stack" enough copies of $a$ to reach the value of $b$. I specify positive numbers, because for $a=0$, there is no finite $n$ such that $n(0)>b$ for finite positive $b$. There is no number of zeros you can stack on top of each other to reach a nonzero number.

An infinitesimal number is a number that does not have the Archimedean property. In the real numbers, only zero is infinitesimal. We often say that the real numbers have no infinitesimals, but what we mean is that there are no interesting, nonzero infinitesimals. Given one of these nonzero infinitesimals (we commonly use a symbol such as $\epsilon$), there is no finite number $n$ such that $n\epsilon>1$, or $n\epsilon>a$ where $a$ is a positive finite real number. No matter how many copies of $\epsilon$ you stack, you can't reach 1, or 0.1, or any finite real number, no matter how small the target is.

## Goals and non-goals

Note that I don't expect this to produce a competitor to IEEE-754 or anything like that. Surreal numbers are useful theoretically, but not for actual calculations. Specifically, the amount of information in a number scales with the generation-number of that number. For the interesting numbers (infinites and inifintesimals) as well as for many perfectly ordinary numbers like 1/3, there will be a literally infinite amount of information in the number, an infinitely tall tree of sets where all the leaves are the empty set. 

If you actually want to calculate with infinitesimals, the [Levi-Civita Field](https://en.wikipedia.org/wiki/Levi-Civita_field) may be a better starting-point.

## Python structure

We build this based on a class, built around a left set and right set, sets being the built-in `frozenset` datatype in Python. We use frozen sets to show our intention that a surreal number is is immutable.

## The two rules

> This shall be the first rule: Every number corresponds to two sets of previously created numbers, such that no member of the left set is greater than or equal to any member of the right set.

De-scripturfying this, we get the following formal logic symbols and vanilla English translation. First, Alice and Bob use lower-case letters to represent numbers, and capital letters to represent sets of numbers. A number is an ordered tuple of sets of numbers $x=\left<X_L,X_R\right>$. In general, for any number $x$, we will say that its sets are the left set $X_L$ and the right set $X_R$ (same letter only capitalized), and that the general members of these sets are $x_L\in X_L$ and $x_R\in X_R$. Similarly we will say $y=\left<Y_L,Y_R\right>$ with $y_L\in Y_L$ and $y_R \in Y_R$, and so on with any letter $a$-$z$.

Two sets represent a valid surreal number if and only if:

* $\lnot \exists x_L\in X_L(\forall x_R \in X_R(x_L\ge x_R))$ It is not the case that there is a number $x_L$ in $X_L$ that is greater than or equal to any member $x_R$ of $X_R$


Conway's first rule is checked in the `__init__` constructor, which throws a ValueError if the rule is violated.

> And the second rule shall be this: One number is less than or equal to another number if and only if no member of the first number’s left set is greater than or equal to the second number, and no member of the second number’s right set is less than or equal to the first number.

This one takes a bit of thought, as it is quite concise. We will follow Alice and Bob's convention of showing numbers as lower-case, and sets as upper-case. Let's try to expand it a bit:

We have two numbers, $x=\left<X_L,X_R\right>$ and $y=\left<Y_L,Y_R\right>$. The statement $x\le y$ is true if and only if the folowing two conditions are both true:

* $\lnot\exists x_L \in X_L(x_L\ge y)$  It is not the case that there exists a number in $X_L$ which is greater than or equal to $y$
* $\lnot\exists y_R \in Y_R(y_R\le x)$  It is not the case that there exists a number in $Y_R$ which is less than or equal to $x$

I hate the fact that there is a not in front of both of these, and it also refers to the greater-or-equal operator which we haven't defined yet. But, we have to be super-careful in rewriting the definitions, because the ones given work well with empty sets, which are needed to bootstrap the whole system. I can handle the $\lnot$ in these definitions, but I would really like to get rid of the $\ge$. Intuitively, we would think that if $a\ge b$, then $b\le a$. I think I need this as an axiom, or perhaps a definition of what $\ge$ means. If we accept this, the rules become:

* $\lnot\exists x_L \in X_L(y \le x_L)$  It is not the case that there exists a number in $X_L$ which $y$ is less than or equal to
* $\lnot\exists y_R \in Y_R(y_R\le x)$  It is not the case that there exists a number in $Y_R$ which is less than or equal to $x$

We can do the same thing with the first rule -- A number is valid if and only if:

* $\lnot \exists x_L\in X_L(\forall x_R \in X_R(x_R\le x_L))$ (It is not the case that there is a number $x_L$ in $X_L$ such that any member $x_R$ of $X_R$ is less than or equal to $x_L$)

Conway's second rule is implemented with the `__le__` magic function, which will allow us to use code such as `a<=b`. Our definition of $\ge$ allows us to implement `__ge__` in terms of `__le__`.

## Equality, alikeness, and identity
One of the things to be careful of is equality: Two surreal numbers $a$ and $b$ are equal if and only if both $a \le b$ and $b \le a$. Note that this does not imply that the sets composing $a$ and $b$ are equal, IE it is plausible two surreal numbers can have the same value, much as two rational numbers may have the same value, even though they have different numerators and denominators: $\frac{1}{2}=\frac{2}{4}$. 

Because of this, we have two notions of equality: One is the Python notion, where if two values compare equal with the `==` operator, they should have the same `hash()` value. This is required in order for things like sets to work, and we want sets to work because we are building these surreal numbers upon them. The other is the surreal equality implied by both $a \le b$ and $b \le a$ being true. In order to properly implement the `__eq__` operator, we need to either decide that two numbers with the same value are equal, and therefore must give the same hash, or are not, and therefore can't use `==`. If we do the first, it becomes impossible to put two different numbers with the same value into a set, equivalent to if we had rational numbers and tried to make a set out of $\{1/2,2/4\}$. If we do the second, it becomes impossible to say $1/2=2/4$. Instead we would have to say $1/2\le 2/4$ and $2/4 \le 1/2$.

This is what Alice and Bob mean by *similar to* or *like*, and once we get around to converting surreal numbers to real numbers, I anticipate that two surreal numbers which are alike will convert to the same real number. We define similarity ($=$ in math, `//` in code) as: Two numbers are alike only if each is less than or equal to the other, and they are not alike if this is not true. I haven't finished the book yet, but I haven't seen Alice and Bob refer to the other kind of equals, which I will refer to as *identity*, IE two numbers are identical if their left sets have the same members and their right sets have the same members. There won't be a single math symbol for identity -- if we need to refer to it, we will say $(X_L=Y_L)\land(X_R=Y_R)$ where the $=$ sign there operates on two sets and therefore refers to set equality, not number equality.

Identity isn't quite the right word, either, since Python uses that word to mean two references to the same object in the same place in memory. However, I don't need that concept in surreal number math.

$$\begin{eqnarray*}
x= y&\equiv&(x\le y)\land(y\le x) \\
x\ne y&\equiv&\lnot(x=y)\\
\end{eqnarray*}$$

It might be a good idea to steal the `//` operator, since integer division does not seem to be a natural operator for surreal numbers, and the parallel lines remind us of the parallel lines in an equals sign. The operator would return a boolean, so we could say:

`if a//b:
     print("a and b are alike")`
     
This will preserve the meaning of `==` as matched up with `hash()`, and will allow two different surreal representations of the same real number to go together in a set. 

We need a matching operator for not alike, and will steal `%` since again I don't think that modulo will be a useful surreal operator, and this one has a slash through it like $\ne$.

Now that we have these operators, we can define $\lt$ and $\gt$:

$$\begin{eqnarray*}
x \lt y&\equiv&(x\le y) \land \lnot(y\le x) \\
x \gt y&\equiv&y \lt x 
\end{eqnarray*}$$

In [ ]:
class Surreal:
    def __init__(self,X_L,X_R,verbose=False):
        #This shall be the first rule: Every number corresponds to two
        #sets of previously created numbers, such that no member of the
        #left set is greater than or equal to any member of the right set.
        for x_L in X_L:
            for x_R in X_R:
                if x_R<=x_L:
                    if verbose:
                        print("x_R ("+str(x_R)+")<=x_L ("+str(x_L)+")")
                    raise ValueError("Not a valid surreal number")
        self.L=frozenset(X_L)
        self.R=frozenset(X_R)
    def __le__(x,y):
        #And the second rule shall be this: One number is less than or
        #equal to another number if and only if no member of the first
        #number’s left set is greater than or equal to the second number,
        #and no member of the second number’s right set is less than or
        #equal to the first number.

        #Part 1 - "no member of the first number’s left set is greater
        #than or equal to the second number"
        for x_L in x.L:
            if y<=x_L:
                return False
        #Part 2 - "no member of the second number’s right set is less
        #than or equal to the first number"
        for y_R in y.R:
            if y_R<=x:
                return False
        return True
    def __ge__(x,y):
        """Define x>=y as y<=x"""
        return y<=x
    def __lt__(x,y):
        return x<=y and not y<=x
    def __gt__(x,y):
        return y<x
    def __floordiv__(x,y):
        """Steal the // operator for surreal alikeness"""
        return (x<=y) and (y<=x)
    def __mod__(x,y):
        """Steal the % operator for surreal non-likeness"""
        return not x//y
    def __str__(self):
        #Symbol for day 1 number . (zero)
        if len(self.L)==0 and len(self.R)==0:
            return "."
        #Symbols for day 2 numbers | (+1) and - (-1)
        if self.L==Surreal.p1.L and self.R==Surreal.p1.R:
            return "|"
        if self.L==Surreal.m1.L and self.R==Surreal.m1.R:
            return "-"
        result="<"
        for x_L in self.L:
            result+=str(x_L)
        result+=":"
        for x_R in self.R:
            result+=str(x_R)
        result+=">"
        return result
    def __repr__(self):
        return "Surreal("+repr(self.L)+","+repr(self.R)+")"


## The First Day
> And the first number was created from the void left set and the void right set. Conway called this number "zero," and said that it shall be a sign to separate positive numbers from negative numbers. Conway proved that zero was less than or equal to zero, and he saw that it was good. And the evening and the morning were the day of zero. 

According to the first rule, every number is composed of sets of previously created numbers. So, where do we get any numbers to build upon? We use the magic of the empty set. We can define a number with a set of *no* numbers on the left, and a set of *no* numbers on the right. We will eventually be able to evaluate (give a conventional "real number" value to) many of our numbers, so we will be able to identify this with real number zero, but for now we will just use the stone symbols:

$$\bullet=\left<:\right>$$

or more conventionally:

$$\begin{eqnarray*}
\emptyset&=&\{\} \\
0&=&\left<\emptyset,\emptyset\right> \\
 &=&\left<\{\},\{\}\right>
\end{eqnarray*}$$

We can talk out our proof that zero is in fact less than or equal to zero, and then we can check that our code above has the same result. From the two parts of rule 2:

$x\le y$ is true if and only if the folowing two conditions are both true:

* $\lnot\exists x_L \in X_L(x_L\ge y)$  It is not the case that there exists a number in $X_L$ which is greater than or equal to $y$
* $\lnot\exists y_R \in Y_R(y_R\le x)$  It is not the case that there exists a number in $Y_R$ which is greater than or equal to $x$

There are no numbers at all in set $0_L$, so there is not a number that is greater than or equal to $0$. Similarly, there are no numbers in set $0_R$, so there is not a number that is less than or equal to $0$. 

In code, we tuck this zero into a static field of the Surreal class. We can't do this in any equivalent to `__init__` or class `__init__`, because we need `__init__` to already be avaliable. So we do it outside. 

In [ ]:
Surreal.zero=Surreal({},{})
print("Conventional form: ",str(Surreal.zero))
print("Representaion form: ",repr(Surreal.zero))
print("0<=0",Surreal.zero<=Surreal.zero) #Should be True
print("0>=0",Surreal.zero>=Surreal.zero) #Should be True
print("0//0",Surreal.zero//Surreal.zero) #Check for alikeness, should be True
print("0 %0",Surreal.zero %Surreal.zero) #Check for non-likeness, should be False
print("0 >0",Surreal.zero >Surreal.zero) #Check for greater than, should be False
print("0 <0",Surreal.zero <Surreal.zero) #Check for less than, should be False

## The Second Day

>On the next day, two more numbers were created, one with zero as its left set and one with zero as its right set. And Conway called the former number "one," and the latter he called "minus one." 

Our amount of work is increasing exponentially. To construct a generation of numbers, take the [Power set](https://en.wikipedia.org/wiki/Power_set) of all the numbers in the previous generation, then match them up with each other, check them against rule 1, and keep the pairs that are valid numbers.

In [ ]:
def powerset(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]

from itertools import product        
        
def next_day(prev_days,verbose=False):
    numbers=set()
    #Just construct every permutation of the previous days
    for X_L,X_R in product(powerset(prev_days),powerset(prev_days)):
        try:
            number=Surreal(X_L,X_R,verbose=verbose)
        except:
            print("Rejecting invalid number: <",str(X_L),",",str(X_R),">")
        else:
            print("Constructing valid number: ",str(number))
            numbers.add(number)
    return numbers

In [ ]:
Surreal.p1=Surreal({Surreal.zero},{})
Surreal.m1=Surreal({},{Surreal.zero})

day2=next_day((Surreal.zero,),verbose=True)
for number in day2:
    print(str(number))


Note that this gives us three numbers, since it reconstructs zero by matching up the empty set element of the powerset with itself. In general, the code above will reconstruct all previous generation numbers, as well as the new numbers in this generation.

In stone notation, we have:

$$\begin{eqnarray*}
|&=&\left<\bullet:\right> \\
-&=&\left<:\bullet\right> \\
\end{eqnarray*}$$

Now on to the proofs: 

> And he proved that minus one is less than but not equal to zero and zero is less than but not equal to one.

### -1<0

The text says that we prove that $\left<\{\},\{0\}\right> \lt \left<\{\},\{\}\right>$, which is the same as proving both:

$$\begin{eqnarray*}
\left<\{\},\{0\}\right>&\le&\left<\{\},\{\}\right> \\
\left<\{\},\{0\}\right>&\ne&\left<\{\},\{\}\right> \\
\end{eqnarray*}$$

First, prove that $-1\le 0$: 
* $\lnot\exists x_L \in (-1)_L(0 \le x_L)$  It is not the case that there exists a number in $(-1)_L$ which $0$ is less than or equal to. Since there are no numbers in $(-1)_L$, this is true.
* $\lnot\exists y_R \in 0_R(y_R\le x)$  It is not the case that there exists a number in $0_R$ which is less than or equal to $x$. Since there are no numbers in $0_R$, this is true.

If $x=y$ is false, then one or both of $x\le y$ or $y\le x$ has to be false. Since we just proved above that $-1\le 0$, we focus on the second term, and prove that $0\not\le -1$. When we check $0\le -1$, we find:

* $\lnot\exists x_L \in 0_L(y \le x_L)$  It is not the case that there exists a number in $0_L$ which $-1$ is less than or equal to. Since there are no numbers in $0_L$, this is true.
* $\lnot\exists y_R \in (-1)_R(y_R\le 0)$  It is not the case that there exists a number in $Y_R$ which is less than or equal to $x$. This time there is a number in $(-1)_R$ this time, and it is $0$. Since $0\le 0$, this whole part is false, and $\lnot(0\le -1)$ or equivalently $0\not\le -1$. Since this part is false, $0\ne -1$, and therefore since $-1\le 0$, $-1<0$.

In [ ]:
print("-1<=0",Surreal.m1<=Surreal.zero) #Should be true
print("0<=-1",Surreal.zero<=Surreal.m1) #Should be false
print("-1<0",Surreal.m1<Surreal.zero)   #Should be true


### 0<1

Almost identically, we prove that $\left<\{\},\{\}\right> \lt \left<\{0\},\{\}\right>$, which is the same as proving both:

$$\begin{eqnarray*}
\left<\{\},\{\}\right>&\le&\left<\{0\},\{\}\right> \\
\left<\{\},\{\}\right>&\ne&\left<\{0\},\{\}\right> \\
\end{eqnarray*}$$

The statement $0\le 1$ is true if and only if the folowing two conditions are both true:

* $\lnot\exists x_L \in 0_L(1 \le x_L)$  It is not the case that there exists a number in $0_L$ which $1$ is less than or equal to. This is true since there are no elements in $0_L$.
* $\lnot\exists y_R \in 1_R(y_R\le 0)$  It is not the case that there exists a number in $1_R$ which is less than or equal to $0$. This is true since there are no elements in $1_R$.

Similarly, f $x=y$ is false, then one or both of $x\le y$ or $y\le x$ has to be false. Since we just proved above that $0\le 1$, we focus on the second term, and prove that $1\not\le 0$. When we check $1\le 0$, we find:

* $\lnot\exists x_L \in 1_L(0 \le x_L)$  It is not the case that there exists a number in $1_L$ which $0$ is less than or equal to. In this case, there is a number in $1_L$, $0$. We proved above that $0=0$, so this case is false and we don't need to consider the other case. $1\not\le 0$, so therefore $0\ne 1$ and since $0\le 1$, $0<1$.

In [ ]:
print("0<=1",Surreal.zero<=Surreal.p1) #Should be True
print("1<=0",Surreal.p1<=Surreal.zero) #should be False
print("0<1",Surreal.zero<Surreal.p1) #Should be True

## The Third Day
There are three numbers in the previous generation, so the power set size will be $2^3=8$ and there will be at most $8\times 8=64$ numbers in the next generation. Not all of these will be valid (for instance, any number with the same set on each side isn't. Rather than just throw these all at `__init__`, let's be systematic and see what the pattern of numbers look like. We will take the power set and make a table with 8 rows and 8 columns, each labeled by one element of the power set. It doesn't matter what order the power set is in, but since we have ordering, let's take the power set as an ordered list.


In [ ]:
day2p=sorted(list(powerset(day2)))
for s in day2p:
    for n in s:
        print(str(n).strip()+",",end='')
    print()

In [ ]:
from IPython.display import display, Markdown, Latex
table="<table><thead><tr><th>L\R</th>"
for sR in day2p:
    table+="<th>"
    if len(sR)==0:
        table+="{}"
    else:
        for n in sorted(list(sR)):
            table=table+str(n)
    table+="</th>"
table+="</tr></thead>\n"
for sL in day2p:
    table+="<tr><th>"
    if len(sL)==0:
        table+="{}"
    else:
        for n in sorted(list(sL)):
            table=table+str(n)
    table+="</th>"
    for sR in day2p:
        try:
            n=Surreal(sL,sR,verbose=False)
        except ValueError:
            table+="<td>X</td>"
        else:
            table+="<td><"
            for nL in sorted(list(n.L)):
                table=table+str(nL)
            table+=':'
            for nR in sorted(list(n.R)):
                table=table+str(nR)
            table+="></td>"
    table+="</tr>\n"

table+="</table>"
            
display(Markdown(table))

What do we see from this?

* Making tables programatically is hard
* Any number with an empty set on either side is valid (15 numbers)
* Almost all other numbers are invalid, with only 5 other valid numbers, for a total of 20.

Alice and Bob found the following numbers:
$$
<:>\,<-:>\,<\bullet:>\,<|:>\,<-\bullet:>\,<-|:>\,<\bullet |:>\, \\
<-\bullet|:>\,<:->\,<:\bullet>\,<:|>\,<:-\bullet>\,<:-|>\, \\
<:\bullet|>\,<:-\bullet|>\,<-:\bullet>\,<\bullet:|>\,<-\bullet:|>\,<-:\bullet|>\, \\
<-:|>
$$


We can do better than this, we can produce a sorted list of numbers, and check whether each number is equal to its neighbors, hopefully 